#  Import

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
from sklearn.model_selection import KFold
from functions.common_function import *
from functions.initialize_model import initialize_model_expanded
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from scipy.stats import pearsonr

'''Enabling plotting of graphs just below the plotting commands'''
%matplotlib inline
'''Enabling the disply of all rows and columns within the dataframe'''
pd.set_option("display.max_rows", None, "display.max_columns", None)

#  Constant

In [ ]:
num_feature = 8
cat_col = [4, 5]
num_ori_feature = num_feature - len(cat_col)
num_target = 3
limit = pd.DataFrame({'lower' : [303, 20, 0, 2, 0, 0, 0, 0, 122, 1236, 14], \
                     'higher' : [840, 44, 17, 5, 1, 1, 1, 1, 408, 3240, 101], \
                     'ref' : [530, 40, 14, 3.2, np.nan, np.nan,np.nan, np.nan, np.nan, np.nan, np.nan]})


'''Import dataset'''
dataset = pd.read_csv("Dataset/Choudhury_Dataset.csv")

# Implementation

In [ ]:
'''Converting Categorical Data into binary representation'''
converted_dataset = convert_cat(dataset, cat_col, num_ori_feature, num_target, [dataset.iloc[:, 4].unique(), dataset.iloc[:, 5].unique()])

'''Normalising dataset according to higher and lower limit values'''
normalised_dataset = normalise(converted_dataset, limit)

'''Feature Target Splitting'''
feature, target = x_y_split(normalised_dataset, num_feature, num_target)

'''Model Fitting'''
linear_reg_model = LinearRegression()
MAE = abs(cross_val_score(linear_reg_model, feature, target, cv= 4, scoring= 'neg_mean_absolute_error'))
MSE = abs(cross_val_score(linear_reg_model, feature, target, cv= 4, scoring= 'neg_mean_squared_error' ))
RMSE = abs(cross_val_score(linear_reg_model, feature, target, cv=4, scoring= 'neg_root_mean_squared_error' ))


'''Provide average score'''
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(MAE)):
    print('------------------------------------------------------------------------')
    print(f'> Fold {i+1} - MAE: {MAE[i]} - MSE: {MSE[i]}- RMSE: {RMSE[i]}') 
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> MAE: {np.mean(MAE)} - Standard Deviation: {np.std(MAE)}')
print(f'> MSE: {np.mean(MSE)}')
print(f'> RMSE: {np.mean(RMSE)}')
print('------------------------------------------------------------------------')

In [ ]:
linear_reg_model  = LinearRegression()
X_train, X_test, Y_train, Y_test = train_test_split(feature, target, test_size = 0.20, random_state = 0)
linear_reg_model.fit(X_train, Y_train)
Y_pred = linear_reg_model.predict(X_test)
prediction = pd.DataFrame(Y_pred, columns = get_col_names(target))

'''Preparation to Rescale target values'''
min_y = limit.iloc[num_feature: num_feature + num_target, 0].to_list()
max_y = limit.iloc[num_feature: num_feature + num_target, 1].to_list()

corr_list = []
'''Tabulating the differences of Expected and Predictions made by the ANN Model'''
for i in range(len(X_test)):
    '''Rescaling of normalised data'''
    expected = pd.DataFrame(inverse_transform(Y_test.iloc[i].to_list(), max_y, min_y))
    predicted = pd.DataFrame(inverse_transform(prediction.iloc[i].to_list(), max_y, min_y))
    comparison_df = pd.concat([expected, predicted], axis = 1)
    comparison_df.columns = ['Expected', 'Predicted']
    comparison_df.index = get_col_names(target)
    display(comparison_df.style.set_caption(f"Element {i + 1}"))
    corr, _ = pearsonr(expected.iloc[:, 0].tolist(), predicted.iloc[:, 0].tolist())
    corr_list.append(corr)
    
'''Provide average score'''
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(corr_list)):
    print('------------------------------------------------------------------------')
    print(f'> Iteration {i+1} - Pearson Correlation: {corr_list[i]}') 
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Average Pearsons Correlation: {np.mean(corr_list)} - Standard Deviation: {np.std(corr_list)}')
print('------------------------------------------------------------------------')  